# Tosho Recommender (Inference)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import gensim
import janome
from janome.tokenizer import Tokenizer as ja_tokenizer
import re
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from model import *
    
def load_obj(filename):
    with open(filename, 'rb') as handler:
        return pickle.load(handler)

# Load tokenizer
tokenizer = load_obj('model/tosho_recommender_word_tokenizer.pkl')

# Load Book2Vec (book-level representation)
b2v = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('model/book2vec', binary=False)

# Load book descriptions and titles
text_pd = pd.read_csv('data/tosho_processed_clean.csv.bz2', sep='\t', compression='bz2')

# Load tf.keras model
model = gru_model(
                     embedding_dim=300,
                     dropout_rate=0.209,
                     rnn_unit=194,
                     input_shape=(500,),
                     num_features=20000+1,
                     share_gru_weights_on_book=True,
                     use_attention_on_book=True,
                     use_attention_on_user=True,
                     use_batch_norm=False,
                     is_embedding_trainable=False,
                     final_activation='tanh',
                     final_dimension=392,
                     embedding_matrix=np.zeros((20001, 300)))

model.compile(loss='cosine_similarity',
                  optimizer=Adam(lr=0.0044))

x = np.ones((1, 500))
y = np.ones((1, 392))

model.train_on_batch([x, x, x, x], y)

# Load the state of the old model
model.load_weights('model/tosho_recommender_7')

/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def clean_text(text:str):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s{1,}', '', text)
    
    text = re.sub(r'内容紹介', '', text)
    text = re.sub(r'出版社からのコメント', '', text)
    text = re.sub(r'商品の説明をすべて表示する', '', text)
    text = re.sub(r'内容（「MARC」データベースより）', '', text)
    text = re.sub(r'内容（「BOOK」データベースより）', '', text)

    non_japanese = re.compile(r"[^0-9\-ぁ-ヶ亜-黑ー]")
    text = re.sub(non_japanese, ' ', text)

    return text.strip()

In [4]:
j_tokenizer = ja_tokenizer()

def wakati_reading(text:str):
    tokens = j_tokenizer.tokenize(text.replace("'", "").lower())
    
    exclude_pos = [u'助動詞']
    
    #分かち書き
    tokens_w_space = ""
    for token in tokens:
        partOfSpeech = token.part_of_speech.split(',')[0]
        
        if partOfSpeech not in exclude_pos:
            tokens_w_space = tokens_w_space + " " + token.surface

    tokens_w_space = tokens_w_space.strip()
    tokens_w_space = re.sub(r'\s{2,}', ' ', tokens_w_space)
    
    return tokens_w_space

In [5]:
def preprocess_text(text:str):
    MAX_SEQUENCE_LENGTH = 500
    
    text = clean_text(text)
    text = wakati_reading(text)

    x = tokenizer.texts_to_sequences([text])
    x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
    
    return x

In [6]:
def recommend_book(book_list:list):
    book_vec = model.predict(book_list)
    book_w2v_result = b2v.similar_by_vector(book_vec[0], topn=3)
                  
    similar_book_id_1 = book_w2v_result[0][0]
    similar_book_title_1 = text_pd[text_pd.id == int(similar_book_id_1)]['title'].values[0]
    similar_book_desc_1 = text_pd[text_pd.id == int(similar_book_id_1)]['description_token'].values[0]
    similar_book_id_2 = book_w2v_result[1][0]
    similar_book_title_2 = text_pd[text_pd.id == int(similar_book_id_2)]['title'].values[0]
    similar_book_desc_2 = text_pd[text_pd.id == int(similar_book_id_2)]['description_token'].values[0]
    similar_book_id_3 = book_w2v_result[2][0]
    similar_book_title_3 = text_pd[text_pd.id == int(similar_book_id_3)]['title'].values[0]
    similar_book_desc_3 = text_pd[text_pd.id == int(similar_book_id_3)]['description_token'].values[0]

    print("1st similar book ID: {}".format(similar_book_id_1))
    print("1st similar book title and description:{}: {}".format(similar_book_title_1, similar_book_desc_1))
    print("\n")
    print("2nd similar book ID: {}".format(similar_book_id_2))
    print("2nd similar book title and description:{}: {}".format(similar_book_title_2, similar_book_desc_2))
    print("\n")
    print("3rd similar book ID: {}".format(similar_book_id_3))
    print("3rd similar book title and description:{}: {}".format(similar_book_title_3, similar_book_desc_3))
    print("\n")
    print("word2vec result: {}".format(book_w2v_result))

In [7]:
# 名探偵コナン (1)
book1 = """
▼第1話/平成のホームズ
▼第2話/小さくなった名探偵
▼第3話/仲間はずれの名探偵
▼第4話/6本目の煙突
▼第5話/もう一人の犯人
▼第6話/迷探偵を名探偵に
▼第7話/血ぬられたアイドル
▼第8話/あなたに似た人
▼第9話/不幸な誤解
"""

# おしりたんてい かいとうと ねらわれた はなよめ (おしりたんていファイル 8)
book2 = """
はたして 今回の かいとうUの ねらいは…!?

おしりたんていの事務所に、とつぜんあらわれた
なぞのいらいにん。だが、そのしょうたいをたちどころに
見抜いたわれらがおしりたんてい。なぞの依頼人が
持ち込んだのは、なんとかいとうUからの予告状だった!

いにしえからの、けっこんの儀式にかくされたお宝を
かいとうUからまもるため、今回もおしりたんていの
名推理が冴えわたる。

迷路や、絵探しなど、
おしりたんていといっしょに謎を解きながら、
真実にせまる、本格的推理読み物シリーズです。

今回も、「かいとうと ねらわれた はなよめ」
「おりの なかの けいかく」の2話収録。

何度読んでも発見がある
推理小説の入り口にも最適な
知的好奇心をくすぐる1冊です。
"""

# がっこうのおばけずかん (どうわがいっぱい)
book3 = """
怖いけどおもしろい、「図鑑」という名の童話、「おばけずかん」シリーズ最新刊。「ひょうほんがいこつ」「おんがくしつのベートーベン」「トイレのはなこさん」などなど、毎日通っている学校にもこわ~いおばけはいっぱいいるけど、このお話を読めば、だいじょうぶ!


昔から知られているおばけがいっぱい登場する、「図鑑」という名前の童話「おばけずかん」シリーズの新刊です。
それぞれのおばけが、どんなふうに怖いのか。そうならないためには、どうしたらだいじょうぶなのかを、ユーモラスな短いお話仕立てで紹介しています。
登場するおばけはちょっと怖いけど、ちゃんと対応してあげると、意外になさけなくて、かわいいところもあったりします。
怖くて、笑えて、最後はホッとできる。「こわいけど、おもしろい」、新しいおばけの童話シリーズ第4弾です。
読者に身近な小学校を舞台に、『トイレのはなこさん』のような新しいおばけや、オリジナルのおばけ『れんぞくこうちょうせんせい』も登場。シリーズの新しい展開を見せる一冊です。

●この本に登場するおばけ

ひょうほんがいこつ
おんがくしつの ベートーベン
トイレの はなこさん
こうていの にのみやきんじろう
ゆうれいアナウンサー
れんぞくこうちょうせんせい
みつめの 六ねんせい
まよなかの まぼろしうんどうかい

※漢字は使用しません
"""

# 星のカービィ 虹の島々を救え!の巻
book4 = """
カービィは、リック&カイン&クーと虹の島々を救う大冒険に出発!

カービィの友だち、リック&カイン&クーがやってきた!
三人が住む虹の島々に雨が降らなくなり、困っているらしい。
カービィはメタナイトやデデデ大王と虹の島々へ!!
原因を知る女の子・ピリカと出会い
雲の上に向かうと、そこにいたのは
カービィとも仲良しな友だちの、グーイだった。
なぜか、グーイはカービィたちに襲いかかってきて!?
いったい、何が起こったのか……?
事件の解決に、カービィがいどむ!!
"""
book1 = preprocess_text(book1)
book2 = preprocess_text(book2)
book3 = preprocess_text(book3)
book4 = preprocess_text(book4)

recommend_book([book1, book2, book3, book4])

1st similar book ID: 3900691
1st similar book title and description:おじいちゃんのごくらくごくらく: おじいちゃん は お 風呂 に 入る と いつも 言い ごく らくご くらく この 言葉 は いつしか おじいちゃん と ゆ うた を つなぐ 合言葉 と なり 著者 略歴 著者 紹介 情報 より 西本 鶏 介 奈良 県 生まれ 昭和女子大学 名誉 教授 児童 文学 評論 創作 民話 研究 と 幅広く 活躍 本 データ は この 書籍 が 刊行 さ れ 当時 に 掲載 さ れ て い もの


2nd similar book ID: 1902640
2nd similar book title and description:菜緒のふしぎ物語: 夜 に 大 また で 歩き まわる や しき ぼう 桜 の で 舞い 踊る お ひな 様 たち 少女 と 古い 屋敷 に 住む ふしぎ もの たち と の 出会い 著者 略歴 著者 紹介 情報 より 竹内 もと 代 石川 県 生まれ 奈良 県 橿原 市 在住 近畿大学 農学部 卒業 思議 の 風 ふく 島 小峰 書店 で 第 26 回 日本 児童 文芸 家 協会 賞 第 49 回 産経 児童 出版 文化 賞 フジテレビ 賞 を 受賞 日本 児童 文芸 家 協会 日本 児童 文学 者 協会 会員 プレアデス 同人 こみ ね ゆら 熊本 市 生まれ 東京 芸術 大学 油 画 同 大学院 修 フランス 政府 給費 留学生 として 渡 仏 後 8 年 半 滞在 さくら この たんじょう 日 宮川 ひろ 文 童心 社 で 2005 年 日本 絵本 賞 を 受賞 フランス で も コーザ ベレリ 文 を 出版 本 データ は この 書籍 が 刊行 さ れ 当時 に 掲載 さ れ て い もの


3rd similar book ID: 1902338
3rd similar book title and description:森のおくの小さな物語: 森 に 暮らす 動物 たち の 心 あたたまる 家族 の す がた を 描い 11 の 物語 たとえば コウモリ の 歌 森 の 洞くつ で 暮らす コウモリ たち が 度 も っ こと が ない の は わけ が あ